In [1]:
import numpy as np
import math
from sklearn.utils.extmath import softmax
from import_NB import import_NB
from pprint import pprint

In [2]:
'''Import data and set parameters'''
X, y = import_NB()                       # Input data & labels
d = X.shape[0]                           # Number of dimensions
N = X.shape[1]                           # Number of samples
C = list(set(y))                         # Classes
K = len(C)                               # Number of classes
Nj = []                                  # Number of samples per class
for c in C:
    Nj.append(len(y[y==c]))

In [3]:
'''Maximum Likelihood Estimations'''
PCj = []   # Prior Probabilities
Uj = []    # Class Means
u_Sj = []  # Variances
Sj = []    # Diagonal Covariance Matrix - check how to do this if it's shared
for c, i in zip(C, range(K)):
    PCj.append(Nj[i]/N)
    Uj.append(1/Nj[i] * np.sum(X[:,y==c], axis=1)[:,np.newaxis])
    u_Sj.append(1/Nj[i] * np.sum((X[:,y==c]-Uj[i]) * (X[:,y==c]-Uj[i]), axis=1))
    Sj.append(np.diag(np.diagonal(1/Nj[i] * (X[:,y==c]-Uj[i]) @ (X[:,y==c]-Uj[i]).T)))

(2, 2)


In [5]:
'''Class-conditional distributions'''
# np.set_printoptions(suppress=True)

# u_PXCj = []
# for i in range(K):
#     bruh = 1
#     for n in range(d):
#         bruh *= 1/np.sqrt(2*np.pi*u_Sj[i][n])  *  np.exp(-0.5 * ((X[n,:] - Uj[i][n])**2)/u_Sj[i][n])
#     u_PXCj.append(bruh)
    
PXCj = []    
for i in range(K):
    temp = 1/np.sqrt(np.linalg.det(2*np.pi*Sj[i]))  *  np.exp(-0.5 * (X-Uj[i]).T @ np.linalg.inv(Sj[i]) @ (X-Uj[i]))
    PXCj.append(np.diagonal(temp))

(1000, 1000)


In [39]:
'''Posterior Probabilities'''
PCjX = []
for i in range(K):
    PCjX.append(PCj[i] * PXCj[i])

In [40]:
'''Maximum Posterior Selection'''
y_pred = np.zeros(N)
for n in range(N):
    temp = np.zeros(K)
    for i in range(K):
        temp[i] = PCjX[i][n]
    y_pred[n] = np.argmax(temp)
y_pred = y_pred.astype(int)